In [83]:
## Importing Lib
from pyspark import SparkContext,SQLContext
import pyspark.sql.functions as f
sc =SparkContext.getOrCreate()
sqlcontext=SQLContext(sc)
# rawData = sc.textFile('file.txt')

In [198]:
## Reading CSV
df=sc.textFile('survey.csv')
df1=sqlcontext.read.format("csv").option("header", "true").csv("survey.csv")

In [199]:
sqlContext.createDataFrame(df.map(lambda text: text.split( "," ))).

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string, _16: string, _17: string, _18: string, _19: string, _20: string, _21: string, _22: string, _23: string, _24: string, _25: string, _26: string, _27: string]

In [114]:
## Writing to csv
df1.write.option("header", "true").csv("spark_survey3.csv")

In [40]:
spark.sql("SELECT * FROM survey").show(2)

+-------------+---+------+-------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+-------------+-------------------------+-----------------------+------------+----------+-----------------------+---------------------+------------------+---------------+--------+
|    Timestamp|Age|Gender|      Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|        leave|mental_health_consequence|phys_health_consequence|   coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------+---+------+-------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+-------------+-------------------

In [86]:
df1.registerTempTable('survey')
spark.sql("SELECT * FROM survey").show(2)

+-------------+---+------+-------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+-------------+-------------------------+-----------------------+------------+----------+-----------------------+---------------------+------------------+---------------+--------+
|    Timestamp|Age|Gender|      Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|        leave|mental_health_consequence|phys_health_consequence|   coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------+---+------+-------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+-------------+-------------------

In [87]:
p_df=df1.toPandas()
p_df[p_df.duplicated()]

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
860,8/28/14 17:02,32,male,United Kingdom,NA,No,Yes,Yes,Rarely,25-Jun,...,Don't know,Yes,Maybe,Some of them,No,No,No,No,No,NA


In [88]:
## Finding the dupicated records
df1.groupBy(df1.columns)\
    .count()\
    .where(f.col('count') > 1)\
    .show()

+-------------+---+------+--------------+-----+-------------+--------------+---------+--------------+------------+-----------+------------+--------+------------+----------------+---------+---------+----------+-------------------------+-----------------------+------------+----------+-----------------------+---------------------+------------------+---------------+--------+-----+
|    Timestamp|Age|Gender|       Country|state|self_employed|family_history|treatment|work_interfere|no_employees|remote_work|tech_company|benefits|care_options|wellness_program|seek_help|anonymity|     leave|mental_health_consequence|phys_health_consequence|   coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|count|
+-------------+---+------+--------------+-----+-------------+--------------+---------+--------------+------------+-----------+------------+--------+------------+----------------+---------+---------+----------+-------------------------+-----

In [155]:
spark.sql('SELECT Timestamp,Age,Gender,count(*) FROM survey\
          GROUP BY Timestamp,Age,Gender HAVING count(*) >1').show()

+-------------+---+------+--------+
|    Timestamp|Age|Gender|count(1)|
+-------------+---+------+--------+
|8/28/14 17:02| 32|  male|       2|
|8/27/14 15:25| 29|  Male|       2|
|8/27/14 13:24| 38|  Male|       2|
|8/27/14 12:34| 27|  Male|       2|
| 8/28/14 9:56| 32|  Male|       2|
|8/27/14 15:27| 33|  Male|       2|
+-------------+---+------+--------+



In [140]:
## Using the union
spark.sql("SELECT * FROM survey WHERE 1=1 AND Gender='M'").registerTempTable('m_tb')
spark.sql("SELECT * FROM survey WHERE 1=1 AND Gender='Male'").registerTempTable('male_tb')
spark.sql("SELECT * FROM m_tb UNION ALL SELECT * FROM male_tb").show(5)

+-------------+---+------+-------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+--------------+-------------------------+-----------------------+------------+----------+-----------------------+---------------------+------------------+---------------+--------+
|    Timestamp|Age|Gender|      Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|         leave|mental_health_consequence|phys_health_consequence|   coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------+---+------+-------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+--------------+----------------

In [91]:
## Group By
spark.sql("SELECT treatment,count(*) as ct FROM survey WHERE 1=1 GROUP BY treatment").show()

+---------+---+
|treatment| ct|
+---------+---+
|       No|622|
|      Yes|637|
+---------+---+



In [92]:
spark.sql('SELECT Age,Gender,state FROM m_tb').show(2)

+---+------+-----+
|Age|Gender|state|
+---+------+-----+
| 44|     M|   IN|
| 39|     M|   NA|
+---+------+-----+
only showing top 2 rows



In [93]:
spark.sql('SELECT Age,Gender,state FROM male_tb').show(2)

+---+------+-----+
|Age|Gender|state|
+---+------+-----+
| 32|  Male|   NA|
| 31|  Male|   NA|
+---+------+-----+
only showing top 2 rows



In [115]:
## Join Operation
spark.sql("SELECT m.Age m_age,m.Gender m_gender,m.state m_sate,\
          m1.Age m1_age,m1.Gender m1_gender,m1.state m1_sate \
          FROM m_tb m,male_tb m1 WHERE 1=1 AND m.state=m1.state").show(5)

+-----+--------+------+------+---------+-------+
|m_age|m_gender|m_sate|m1_age|m1_gender|m1_sate|
+-----+--------+------+------+---------+-------+
|   44|       M|    IN|    32|     Male|     IN|
|   44|       M|    IN|    25|     Male|     IN|
|   44|       M|    IN|    48|     Male|     IN|
|   44|       M|    IN|    34|     Male|     IN|
|   44|       M|    IN|    19|     Male|     IN|
+-----+--------+------+------+---------+-------+
only showing top 5 rows



In [95]:
## Aggregate Function
spark.sql("SELECT m.Age ,count(m.state) \
          FROM m_tb m,male_tb m1 WHERE 1=1 AND m.state=m1.state GROUP BY m.Age").show(5)

+---+------------+
|Age|count(state)|
+---+------------+
| 51|         272|
| 54|          49|
| 29|         927|
| 42|          75|
| 30|         360|
+---+------------+
only showing top 5 rows



In [96]:
## Fetching 13 Record
x=spark.sql("SELECT * \
          FROM survey limit 13").collect()
x[-1]

Row(Timestamp='8/27/14 11:33', Age='42', Gender='female', Country='United States', state='CA', self_employed='NA', family_history='Yes', treatment='Yes', work_interfere='Sometimes', no_employees='26-100', remote_work='No', tech_company='No', benefits='Yes', care_options='Yes', wellness_program='No', seek_help='No', anonymity="Don't know", leave='Somewhat difficult', mental_health_consequence='Yes', phys_health_consequence='Yes', coworkers='Yes', supervisor='Yes', mental_health_interview='Maybe', phys_health_interview='Maybe', mental_vs_physical='No', obs_consequence='Yes', comments='NA')

In [139]:
t_df=spark.sql("SELECT * FROM (SELECT *,ROW_NUMBER() OVER (ORDER BY Timestamp) ix\
          FROM survey WHERE 1=1) WHERE ix=13")
t_df.drop('ix').registerTempTable('13rec')
spark.sql('SELECT * FROM 13rec').show()

+-------------+---+------+-------------+-----+-------------+--------------+---------+--------------+------------+-----------+------------+----------+------------+----------------+---------+----------+----------+-------------------------+-----------------------+------------+----------+-----------------------+---------------------+------------------+---------------+--------+
|    Timestamp|Age|Gender|      Country|state|self_employed|family_history|treatment|work_interfere|no_employees|remote_work|tech_company|  benefits|care_options|wellness_program|seek_help| anonymity|     leave|mental_health_consequence|phys_health_consequence|   coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------+---+------+-------------+-----+-------------+--------------+---------+--------------+------------+-----------+------------+----------+------------+----------------+---------+----------+----------+-------------------------+-----------

In [147]:
y=spark.createDataFrame([x[-1]])

In [151]:
y.registerTempTable('temp')
spark.sql('select * FROM temp').show()

+-------------+---+------+-------------+-----+-------------+--------------+---------+--------------+------------+-----------+------------+--------+------------+----------------+---------+----------+------------------+-------------------------+-----------------------+---------+----------+-----------------------+---------------------+------------------+---------------+--------+
|    Timestamp|Age|Gender|      Country|state|self_employed|family_history|treatment|work_interfere|no_employees|remote_work|tech_company|benefits|care_options|wellness_program|seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------+---+------+-------------+-----+-------------+--------------+---------+--------------+------------+-----------+------------+--------+------------+----------------+---------+----------+------------------+-------------------------

In [195]:
dataRead = sc.textFile('survey.csv')
dataRead.take(5)

import csv
import string

def parseLine(line):
    csvdata = csv.reader([line], delimiter=',', quotechar='"')
    for fields in csvdata:
        try:            
            return { "TimeStamp" : fields[0] , "Age": fields[1], "Gender" : fields[2]}
        except:
            return { "TimeStamp" : "TimeStamp", "Age" :"Age", "Gender": "Gender"}


dataRead_data = dataRead.map(parseLine).filter(lambda ufo: ufo['Gender'] != 'Gender')

In [196]:
dataRead_data.toDF().show()

+---+------+-------------+
|Age|Gender|    TimeStamp|
+---+------+-------------+
| 37|Female|8/27/14 11:29|
| 44|     M|8/27/14 11:29|
| 32|  Male|8/27/14 11:29|
| 31|  Male|8/27/14 11:29|
| 31|  Male|8/27/14 11:30|
| 33|  Male|8/27/14 11:31|
| 35|Female|8/27/14 11:31|
| 39|     M|8/27/14 11:32|
| 42|Female|8/27/14 11:32|
| 23|  Male|8/27/14 11:32|
| 31|  Male|8/27/14 11:32|
| 29|  male|8/27/14 11:32|
| 42|female|8/27/14 11:33|
| 36|  Male|8/27/14 11:33|
| 27|  Male|8/27/14 11:33|
| 29|female|8/27/14 11:34|
| 23|  Male|8/27/14 11:34|
| 32|  Male|8/27/14 11:34|
| 46|  male|8/27/14 11:34|
| 36|  Male|8/27/14 11:35|
+---+------+-------------+
only showing top 20 rows

